This notebook is created at GESIS for general public to test M3 model on gender inference from images. You can upload your own image or use a url. All data is deleted afterwards.

In [ ]:
# #############################################################################
# Importing necessary libraries

# import numpy as np
# import pandas as pd
# import requests as requests
# import json
# from pathlib import Path
# from m3inference import M3Inference
# from collections import OrderedDict
# import pprint
# import urllib.request
# import random
# import re
# import json
# import os
# from PIL import Image
# import csv 
# import sys
# import string 
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#import zipfile as zf


from ipywidgets import TwoByTwoLayout, VBox
import ipywidgets as widgets
import requests as requests
from IPython.display import clear_output
from PIL import Image
import sys
from m3inference import M3Inference
from io import BytesIO
import json
import pprint
import urllib.request
import os

## You can either upload a photo from your computer OR insert a url path to it. Optionally you can also type firstname as M3 uses both textual and visual inputs for better predictions

In [ ]:
'''Set up layout for upload widgets'''
def initialize_widgets():
    global upload, link, name, layout
    upload = widgets.FileUpload(
        accept='image/*', 
        multiple=False  
    )
    link = widgets.Text(
        placeholder='url to an image file',
        description='Link:',
        disabled=False
    )
    name = widgets.Text(
        placeholder='optional',
        description='First name:',
        disabled=False
    )
    
    layout = TwoByTwoLayout(top_left=VBox([upload, link]),
                   top_right=submit,
                   bottom_left=name,
                   bottom_right=clear)
    display(layout)
    
global clear, submit
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

''' TODO Clear the output and delete the files'''
def on_clear_clicked(b):
    os.remove(filename)
    clear_output()
    initialize_widgets()

'''Resize and convert images for M3. Save the file (is deleted afterwards)'''    
def preprocess_and_save(content):
    global filename
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    image = image.resize((224, 224))
    if name.value:
        filename = name.value + '.jpg'
    else:
        filename = "testfile.jpg"
    image.save(filename)

'''Submitting an image either as fileupload or from url'''
def on_submit_clicked(b):
    global content
    if upload.value and link.value:
        print('Please choose only one method for input')
        return False
    elif upload.value:
        uploaded_filename = next(iter(upload.value))
        content = upload.value[uploaded_filename]['content']
    elif link.value:
        content = requests.get(link.value).content
    else:
        print('No image has been uploaded')
        return False
    print('An image you submitted:')
    display(widgets.Image(value=content, width=200))
    preprocess_and_save(content)

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)


In [ ]:
# Running the M3 inference and getting the gender
def Mi_inference():
    if name.value:
        firstname = name.value
    else:
        firstname = ""
    data= {
        "description":"", 
        "id": "",
        "img_path": filename, 
        "lang": "en", 
        "name": firstname, 
        "screen_name": ""
    }
    Json_file='data.jsonl'        # Path to create the Json file
    with open(Json_file, 'w') as json_file:
        json.dump(data, json_file) 
    try:                                            # Here Based on values in output will select the geneder
        m3 = M3Inference()                          # see docstring for details
        pred = m3.infer(Json_file)                  # also see docstring for details
        disc=pprint.pprint(pred)
        main=dict(pred).get("")
        main_gender=main.get('gender')
        main_gender_female=main_gender.get('female')
        main_gender_male=main_gender.get('male')
        if main_gender_female > main_gender_male :
            return('Female')
        elif main_gender_female < main_gender_male :
            return('Male')
        else:
            return('Ambiguity')
    except urllib.error.HTTPError as e:
        print (e)

try:
    gender = Mi_inference()
    print("Predicted gender for your input is: " + gender)
except:
    print("Oops!",sys.exc_info()[0],"occured.")



In [ ]:
for filename in os.listdir(sys.path[0]+'/'):
    if filename.endswith(".ipynb") or filename.endswith(".ipynb_checkpoints"):
        pass
    else:
        os.remove(filename)